In [46]:
import pandas as pd

df = pd.read_csv('./data/sales.csv')

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [48]:
df.dropna(how='all', inplace=True)



In [49]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='mixed', errors='coerce')
df['Order Date'] = df['Order Date'] + pd.DateOffset(years=4)


In [50]:
df.to_csv('./data/sales_new.csv', index=False)

In [53]:
df = df[df['Quantity Ordered'].str.strip() != 'Quantity Ordered']
df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'], errors='raise')




In [62]:
df = df[df['Price Each'].str.strip() != "Price Each"]

In [63]:
df['Price Each'] = pd.to_numeric(df['Price Each'])

In [66]:
df.to_csv('./data/sales_new.csv',index=False)

In [70]:


df = pd.read_csv('./data/sales_with_coords.csv')



In [71]:
df['city'] = df['Purchase Address'].str.split(',').str[1].str.strip()

df = df[df['Order Date'].dt.year == 2023]

In [73]:
df.to_csv("./data/sales_with_coords.csv", index=False)

## Reorganize dates to make data more random

In [59]:
import pandas as pd
import janitor
import numpy as np

df = pd.read_csv('./data/sales_with_coords_copy.csv')

In [60]:
df['Order Date'] = pd.to_datetime(df['Order Date'])

In [61]:
df = df.clean_names()

In [62]:
df.head()

,order_id,product,quantity_ordered,price_each,order_date,purchase_address,zip,lat,long,city
0,176558,USB-C Charging Cable,2,11.95,2023-04-19 08:46:00,"917 1st St, Dallas, TX 75001",75001,32.9600,-96.8385,Dallas
1,176559,Bose SoundSport Headphones,1,99.99,2023-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",2215,42.3471,-71.1027,Boston
2,176560,Google Phone,1,600.00,2023-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",90001,33.9731,-118.2479,Los Angeles
3,176560,Wired Headphones,1,11.99,2023-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",90001,33.9731,-118.2479,Los Angeles
4,176561,Wired Headphones,1,11.99,2023-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",90001,33.9731,-118.2479,Los Angeles


In [63]:
df.city.unique()

array(['Dallas', 'Boston', 'Los Angeles', 'San Francisco', 'Seattle',
       'Atlanta', 'New York City', 'Portland', 'Austin'], dtype=object)

In [64]:
# Assume df is your dataframe
# Define city-specific change rates
change_rates = {
    'Dallas': 0.8,
    'Boston': 0.15,
    'Los Angeles': 0.40,
    'San Francisco': 0.05,
    'Seattle': 0.55,
    'Atlanta': 0.6,
    'New York City': 0.25,
    'Portland': 0.09,
    'Austin': 0.45
}

# Function to randomly adjust dates
def adjust_dates(row, change_rate):
    if np.random.rand() < change_rate:  # Only change the date with a certain probability
        new_month = np.random.randint(1, 13)
        try:
            return row['order_date'].replace(month=new_month)
        except ValueError:
            # Adjust the day for the month if out of range
            last_day = pd.Timestamp(year=row['order_date'].year, month=new_month, day=1) + pd.DateOffset(months=1) - pd.DateOffset(days=1)
            return last_day
    else:
        return row['order_date']

# Apply changes to the dataframe
df['order_date'] = df.apply(lambda row: adjust_dates(row, change_rates[row['city']]), axis=1)

In [65]:
df.to_csv("./data/sales_with_coords.csv", index=False)

In [66]:
# Filter the DataFrame for Boston sales in June
boston_june_sales = df[(df['city'] == 'Boston') & (df['order_date'].dt.month == 6)]

# Calculate the number of rows to drop
num_rows_to_drop = int(len(boston_june_sales) * 0.6)

# Randomly select the rows to drop
rows_to_drop = boston_june_sales.sample(n=num_rows_to_drop, random_state=42)

# Drop the selected rows from the DataFrame
df = df.drop(rows_to_drop.index)


In [67]:
df.to_csv("./data/sales_with_coords.csv", index=False)